### An Exploration into Theano
#### Patrick Huston | Spring 2016

This notebook aims to explore the implementation of a simple neural network supported by Theano, a powerful Python library that allows you to define, optimize, and evaluate mathematical expressions involving multi-dimensional arrays efficiently.

In [1]:
import theano
import theano.tensor as T
import theano.tensor.nnet as nnet
import numpy as np

First, we'll set up our input vector and output scalar in proper Theano syntax. We're going to symbolically define two Theano variables called x and y. We'll build our familiar XOR network with 2 input units (+ a bias), 2 hidden units (+ a bias), and 1 output unit. So our x variable will always be a 2-element vector (e.g. [0,1]) and our y variable will always be a scalar and is our expected value for each pair of x values.

In [4]:
x = T.dvector()
y = T.dscalar()

Now let's define a Python function that will be a matrix multiplier and sigmoid function, so it will accept an x vector (and concatenate in a bias value of 1) and a w weight matrix, multiply them, and then run them through a sigmoid function. Theano has the sigmoid function built in the nnet class that we imported above. We'll use this function as our basic layer output function.

In [5]:
def layer(x, w):
    b = np.array([1], dtype=theano.config.floatX)
    new_x = T.concatenate([x, b])
    m = T.dot(w.T, new_x) #theta1: 3x3 * x: 3x1 = 3x1 ;;; theta2: 1x4 * 4x1
    h = nnet.sigmoid(m)
    return h

In order to concatenate a scalar value of 1 to our 1-dimensional vector x, we create a numpy array with a single element (1), and explicitly pass in the dtype parameter to make it a float64 and compatible with our Theano vector variable.

Let's go ahead and implement our gradient descent function. This is where the power of Theano truly shines.  We're just going to have a function that defines a learning rate alpha and accepts a cost/error expression and a weight matrix. It will use Theano's grad() function to compute the gradient of the cost function with respect to the given weight matrix and return an updated weight matrix.

In [6]:
def grad_desc(cost, theta):
    alpha = 0.1 #learning rate
    return theta - (alpha * T.grad(cost, wrt=theta))

At this point, let's initialize some initial weight matrices with Theano. Since our weight matrices will take on definite values, they're not going to be represented as Theano variables, they're going to be defined as Theano's shared variable. A shared variable is what we use for things we want to give a definite value but we also want to update. Notice that I didn't define the alpha or b (the bias term) as shared variables, I just hard-coded them as strict values because I am never going to update/modify them.

In [14]:
theta1 = theano.shared(np.array(np.random.rand(3,3), dtype=theano.config.floatX)) # randomly initialize
theta2 = theano.shared(np.array(np.random.rand(4,1), dtype=theano.config.floatX))

So here we've defined our two weight matrices for our 3 layer network and initialized them using numpy's random class. Again we specifically define the dtype parameter so it will be a float64, compatible with our Theano `dscalar` and `dvector` variable types.

Here's where the fun begins. We can start actually doing our computations for each layer in the network. Of course we'll start by computing the hidden layer's output using our previously defined layer function, and pass in the Theano `x` variable we defined above and our `theta1` matrix.

In [15]:
hid1 = layer(x, theta1) #hidden layer

We can do the same for our final output layer. Notice how we use the `T.sum()` function on the outside which is the same as numpy's sum(). This is only because Theano will complain if you don't make it explicitly clear that our output is returning a scalar and not a matrix. Our matrix dimensional analysis is sure to return a `1x1` single element vector but we need to convert it to a scalar since we're substracting out 1 from y in our cost expression that follows.

In [16]:
out1 = T.sum(layer(hid1, theta2)) #output layer
fc = (out1 - y)**2 #cost expression

Almost there! We're going to compile two Theano functions. One will be our cost expression (for training), and the other will be our output layer expression (to run the network forward).

In [17]:
cost = theano.function(inputs=[x, y], outputs=fc, updates=[
        (theta1, grad_desc(fc, theta1)),
        (theta2, grad_desc(fc, theta2))])
run_forward = theano.function(inputs=[x], outputs=out1)

This Theano function call looks a bit different from the first example - the additional updates parameter is included here. `updates` allows us to update shared variables according to an expression. `updates` expects a list of two tuples:

```
updates=[(shared_variable, update_value), ...]
```

The second part of each tuple can be an expression or function that returns the new value we want to update the first part to. In our case, we have two shared variables we want to update, theta1 and theta2 and we want to use our grad_desc function to give us the updated data. Of course our grad_desc function expects two arguments, a cost function and a weight matrix, so we pass those in. fc is our cost expression. So every time we invoke/call the cost function that we've compiled with Theano, it will also update our shared variables according to our grad_desc rule. Pretty convenient!
Additionally, we've compiled a run_forward function just so we can run the network forward and make sure it has trained properly. We don't need to update anything there.
Now let's define our training data and setup a for loop to iterate through our training epochs.

In [19]:
inputs = np.array([[0,1],[1,0],[1,1],[0,0]]).reshape(4,2) #training data X
exp_y = np.array([1, 1, 0, 0]) #training data Y
cur_cost = 0
for i in range(10000):
    for k in range(len(inputs)):
        cur_cost = cost(inputs[k], exp_y[k]) #call our Theano-compiled cost function, it will auto update weights
    if i % 500 == 0: #only print the cost every 500 epochs/iterations (to save space)
        print('Cost: %s' % (cur_cost,))

Cost: 0.617206144089
Cost: 0.233638561492
Cost: 0.209360719526
Cost: 0.100529080459
Cost: 0.0376772520412
Cost: 0.0328228031737
Cost: 0.0167141782909
Cost: 0.00975994673185
Cost: 0.00665756617065
Cost: 0.00498593881828
Cost: 0.00395956235457
Cost: 0.00327131243897
Cost: 0.00278017971549
Cost: 0.00241324838702
Cost: 0.00212930624461
Cost: 0.00190341066436
Cost: 0.00171962742956
Cost: 0.00156732257637
Cost: 0.00143913908324
Cost: 0.00132983108531


In [20]:
#Training done! Let's test it out
print(run_forward([0,1]))
print(run_forward([1,1]))
print(run_forward([1,0]))
print(run_forward([0,0]))

0.970755280855
0.0303932468439
0.970320060519
0.0351420873692
